In [1]:
import os
import random
import shutil

In [2]:
TestPath = 'data/test1'
TrainPath = 'data/train'

DatasetTrain = 'dataset/train'
DatasetValidation = 'dataset/validation'

In [3]:
def moveFiles():
    files = os.listdir(TrainPath)
    id = 0
    ct = 0
    for file in files:
        src = os.path.join(TrainPath, file)
        num = random.random()
        if num >= 0.8:
            ct += 1
            if 'cat' in str(file):
                tar = os.path.join(DatasetValidation, 'cats', file)
            else:
                tar = os.path.join(DatasetValidation, 'dogs', file)
        else:
            if 'cat' in str(file):
                tar = os.path.join(DatasetTrain, 'cats', file)
            else:
                tar = os.path.join(DatasetTrain, 'dogs', file)
        shutil.copyfile(src, tar)
        id += 1
        if id % 1000 == 0:
            print('%d files moved, %d train, %d validation' % (id, id - ct, ct))

In [4]:
def checkFiles():
    print(len(os.listdir(os.path.join(DatasetTrain, 'cats'))))
    print(len(os.listdir(os.path.join(DatasetTrain, 'dogs'))))
    print(len(os.listdir(os.path.join(DatasetValidation, 'cats'))))
    print(len(os.listdir(os.path.join(DatasetValidation, 'dogs'))))

In [5]:
checkFiles()

10007
9993
2493
2507
